
# Modelo Clasificador de texto
## Violento, No violento
### Busca crear un etiquetado de un violentometro para establecer cuales fraces dichas son violentas dentro de un chat completo
**La violencia dentro de una relacion; buscamos realizar un modelo medidor de violencia de pareja, asi que con mensajes de texto previamente etiquetados como violento o no violento buscamos un modelo clasificador de texto**


# Importaciones

In [ ]:
# Data Carga
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Preparación de datos
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Carga de Datos

In [ ]:
df = pd.read_csv("EntrenamientoData.csv")

# Limpieza

In [ ]:
df.shape

(1001, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        1001 non-null   int64 
 1   Text      1001 non-null   object
 2   Etiqueta  1001 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
df=df.drop('ID', axis=1)

### Eliminacion de Puntuaciones

In [ ]:
import re

# Definir la expresión regular para eliminar la puntuación
regex_puntuacion = re.compile('[^\w\s]')

# Aplicar la expresión regular a la columna de texto
df['Text'] = df['Text'].apply(lambda x: regex_puntuacion.sub('', x))

# Imprimir el dataframe resultante
df

,Text,Etiqueta
0,Yo me encuentro mal pero no te preocupes tú sa...,Violento
1,Me hace muchísima ilusión hacer esto contigo p...,Violento
2,No te encuentras bien Pues Si no acabas este i...,Violento
3,Tengo a mi madre enferma si no me ayudas y pie...,Violento
4,Nadie te entiende como yo,Violento
...,...,...
996,Sigue enfermita mi bb,No violento
997,estoy enferma,No violento
998,Me muero de frio,No violento
999,comigo,No violento


### Eliminar acentos

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from unidecode import unidecode

In [ ]:
df['Text'] = df['Text'].apply(lambda x: unidecode(regex_puntuacion.sub('', x)))

In [ ]:
df

,Text,Etiqueta
0,Yo me encuentro mal pero no te preocupes tu sa...,Violento
1,Me hace muchisima ilusion hacer esto contigo p...,Violento
2,No te encuentras bien Pues Si no acabas este i...,Violento
3,Tengo a mi madre enferma si no me ayudas y pie...,Violento
4,Nadie te entiende como yo,Violento
...,...,...
996,Sigue enfermita mi bb,No violento
997,estoy enferma,No violento
998,Me muero de frio,No violento
999,comigo,No violento


### convertir a minusculas

In [ ]:
df['Text'] = df['Text'].apply(lambda x: x.lower())

In [ ]:
df

,Text,Etiqueta
0,yo me encuentro mal pero no te preocupes tu sa...,Violento
1,me hace muchisima ilusion hacer esto contigo p...,Violento
2,no te encuentras bien pues si no acabas este i...,Violento
3,tengo a mi madre enferma si no me ayudas y pie...,Violento
4,nadie te entiende como yo,Violento
...,...,...
996,sigue enfermita mi bb,No violento
997,estoy enferma,No violento
998,me muero de frio,No violento
999,comigo,No violento


### Eliminar las stopworlds el/la etc

In [ ]:
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def limpiar_texto(texto):
    palabras_vacias = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras_sin_vacias = [palabra for palabra in palabras if palabra not in palabras_vacias]
    texto_sin_vacias = ' '.join(palabras_sin_vacias)
    return texto_sin_vacias

In [ ]:
df['Text'] = df['Text'].apply(limpiar_texto)

In [ ]:
df

,Text,Etiqueta
0,encuentro mal preocupes sal amigos quedo aqui ...,Violento
1,hace muchisima ilusion hacer contigo si quiere...,Violento
2,encuentras bien pues si acabas informe afectar...,Violento
3,madre enferma si ayudas pierdo trabajo podre p...,Violento
4,nadie entiende,Violento
...,...,...
996,sigue enfermita bb,No violento
997,enferma,No violento
998,muero frio,No violento
999,comigo,No violento


### Lematizacion de text

In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import stanza
stanza.download('es')

INFO:stanza:Downloading default packages for language: es (Spanish) ...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')

def lematizar(texto):
    doc = nlp(texto)
    palabras_lematizadas = [word.lemma for sent in doc.sentences for word in sent.words]
    texto_lematizado = ' '.join(palabras_lematizadas)
    return texto_lematizado

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
df['Text'] = df['Text'].apply(lematizar)

In [ ]:
df.head(1)

,Text,Etiqueta
0,encuentro mal preocupar sal amigo quedar aqui ...,Violento


# Naive bayes

### prueba y entrenamiento

### Vectorizacion

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el DataFrame 'df' en conjuntos de entrenamiento y prueba
# 'df['Text']' contiene las características (entradas) del modelo
# 'df['Etiqueta']' contiene las etiquetas (salidas) del modelo
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Etiqueta'], test_size=0.3, random_state=42)

# Descripción de los parámetros utilizados:
# - test_size=0.3: Utiliza el 30% de los datos para la prueba y el 70% para el entrenamiento.
# - random_state=42: Establece una semilla para el generador de números aleatorios para garantizar
#   que la división de los datos sea reproducible. Cada vez que se ejecute este código con la misma
#   semilla (42 en este caso), se obtendrá la misma división de los datos.

# El resultado de la división son cuatro conjuntos de datos:
# - X_train: Contiene el 70% de los datos de la columna 'Text' para el entrenamiento.
# - X_test: Contiene el 30% de los datos de la columna 'Text' para la prueba.
# - y_train: Contiene el 70% de los datos de la columna 'Etiqueta' para el entrenamiento.
# - y_test: Contiene el 30% de los datos de la columna 'Etiqueta' para la prueba.

In [ ]:
# Crear una instancia de CountVectorizer con un número máximo de características (max_features)
vectorizer = CountVectorizer(max_features=698)

# Ajustar el vectorizador a los datos de entrenamiento (X_train) y transformarlos en una matriz de términos-frecuencia
X_train_vec = vectorizer.fit_transform(X_train)

# Transformar los datos de prueba (X_test) en una matriz de términos-frecuencia usando el vectorizador ya ajustado
X_test_vec = vectorizer.transform(X_test)

# Resultados:
# - X_train_vec: Una matriz de términos-frecuencia (sparse matrix) que representa los textos de entrenamiento.
# - X_test_vec: Una matriz de términos-frecuencia (sparse matrix) que representa los textos de prueba.

In [ ]:
X_train_vec

<700x698 sparse matrix of type '<class 'numpy.int64'>'
	with 1944 stored elements in Compressed Sparse Row format>

In [ ]:
# Crear una instancia del clasificador Multinomial Naive Bayes
clf = MultinomialNB()

# Ajustar el clasificador a los datos de entrenamiento vectorizados (X_train_vec) y las etiquetas de entrenamiento (y_train)
clf.fit(X_train_vec, y_train)
# Resultados:
# - clf: Un modelo de Multinomial Naive Bayes ajustado a los datos de entrenamiento, que puede ser utilizado para hacer
#   predicciones sobre nuevos datos.

MultinomialNB()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_vec, y_train)
# Evaluar el modelo ajustado usando los datos de prueba vectorizados (X_test_vec) y las etiquetas de prueba (y_test)
accuracy = clf.score(X_test_vec, y_test)

# Imprimir la exactitud del modelo
print("Exactitud del modelo:", accuracy)
# Resultados:
# - accuracy: Un valor numérico que representa la exactitud del modelo, es decir, la proporción de predicciones
#   correctas realizadas por el modelo sobre los datos de prueba.

Exactitud del modelo: 0.7475083056478405


In [ ]:
# Importar las funciones y clases necesarias de scikit-learn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Vectorizar todo el conjunto de datos 'df' ('Text' y 'Etiqueta')
X = vectorizer.fit_transform(df['Text'])
y = df['Etiqueta']

# Definir el número de particiones (folds) para la validación cruzada
k = 10

# Crear una instancia de KFold para dividir los datos en k particiones, con barajado y una semilla aleatoria para reproducibilidad
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Realizar la validación cruzada y calcular la exactitud para cada partición (fold)
scores = cross_val_score(clf, X, y, cv=kf, error_score='raise')

# Imprimir la exactitud para cada partición (fold)
for i, score in enumerate(scores):
    print("Fold", i+1, "Accuracy:", score)

# Calcular e imprimir la exactitud media a través de todas las particiones (folds)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

# Resultados:
# - scores: Un array de valores numéricos que representan la exactitud del modelo en cada partición de la validación cruzada.
# - mean_accuracy: Un valor numérico que representa la exactitud media del modelo a través de todas las particiones de la validación cruzada.

Fold 1 Accuracy: 0.7920792079207921
Fold 2 Accuracy: 0.79
Fold 3 Accuracy: 0.74
Fold 4 Accuracy: 0.79
Fold 5 Accuracy: 0.76
Fold 6 Accuracy: 0.77
Fold 7 Accuracy: 0.83
Fold 8 Accuracy: 0.77
Fold 9 Accuracy: 0.77
Fold 10 Accuracy: 0.72
Mean Accuracy: 0.7732079207920791


In [ ]:
# Importar la función confusion_matrix del módulo metrics de scikit-learn
from sklearn.metrics import confusion_matrix

# Predecir las etiquetas para los datos de prueba (X_test_vec) usando el clasificador ajustado
y_pred = clf.predict(X_test_vec)

# Calcular la matriz de confusión usando las etiquetas reales de prueba (y_test) y las etiquetas predichas (y_pred)
cm = confusion_matrix(y_test, y_pred)

# Imprimir la matriz de confusión
print("Matriz de confusión:\n", cm)

Matriz de confusión:
 [[116  36]
 [ 40 109]]


In [ ]:
# Importar la función dump del módulo joblib
import joblib

# Guardar el modelo ajustado (clf) en un archivo llamado 'violentometro.joblib'
joblib.dump(clf, 'violentometro.joblib')

['violentometro.joblib']

### Pruebas

In [ ]:
# Texto de ejemplo
nuevo_texto = "Te amo"

# Quitar acentos y símbolos de puntuación
nuevo_texto = unidecode(regex_puntuacion.sub('', nuevo_texto.lower()))

# Convertir el texto a minúsculas
nuevo_texto = nuevo_texto.lower()

# Limpiar stopwords
nuevo_texto = limpiar_texto(nuevo_texto)

# Lematizar texto
nuevo_texto = lematizar(nuevo_texto)

In [ ]:
# Vectorizar el nuevo texto procesado
nuevo_texto_vec = vectorizer.transform([nuevo_texto])

In [ ]:
# Realizar la predicción sobre el nuevo texto vectorizado
y_pred = clf.predict(nuevo_texto_vec)
print("Predicción de clase:", y_pred)

# Obtener las probabilidades predichas para cada clase
y_pred_proba = pd.DataFrame(clf.predict_proba(nuevo_texto_vec), columns=['proba_clase_0', 'proba_clase_1'])
print("Probabilidades por clase:")
print(y_pred_proba)

['Violento']


,proba_clase_0,proba_clase_1
0,0.229127,0.770873


#Modelo LSTM


In [ ]:
df

,ID,Text,Etiqueta
0,1,encuentro mal preocupar sal amigo quedar aqui ...,Violento
1,2,hacer muchisimo ilusion hacer tú si querer dejal,Violento
2,3,encontrar bien pues si acabar informe afectar ...,Violento
3,4,madre enferma si ayuda perder trabajo poder pa...,Violento
4,5,nadie entender,Violento
...,...,...,...
996,997,seguir enfermito bb,No violento
997,998,enferma,No violento
998,999,morir frio,No violento
999,1000,comigo,No violento


In [ ]:
df['Etiqueta'] = [1 if label == "Violento" else 0 for label in df['Etiqueta']]

In [ ]:
df

,ID,Text,Etiqueta
0,1,encuentro mal preocupar sal amigo quedar aqui ...,1
1,2,hacer muchisimo ilusion hacer tú si querer dejal,1
2,3,encontrar bien pues si acabar informe afectar ...,1
3,4,madre enferma si ayuda perder trabajo poder pa...,1
4,5,nadie entender,1
...,...,...,...
996,997,seguir enfermito bb,0
997,998,enferma,0
998,999,morir frio,0
999,1000,comigo,0


# Aplicar El Modelo A Los 5 chats

In [ ]:
df_Pre = pd.read_csv("/content/drive/MyDrive/Modelo Violentometro/5chats_preprocesado.csv")

In [ ]:
df_Pre

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Hour,User,Date,Day_of_Week,Message_Only_Text,Message_Raw,index,Message_Clean,fecha,id_conversacion,preprocesado
0,0,0,0,2,William Salgado,2023-04-10 02:56:44,0,‎Los mensajes y las llamadas están cifrados de...,"('10', '04', '23', '2', '56', '44', 'a.m.', 'W...",0,‎Los mensajes y las llamadas están cifrados de...,2023-04-10,5,mensajes llamadas cifrados extremo extremo nad...
1,1,1,1,2,William Salgado,2023-04-10 02:56:44,0,Si quiera me hubieras avisado que me ibas bor...,"('10', '04', '23', '2', '56', '44', 'a.m.', 'W...",1,Si quiera me hubieras avisado que me ibas bor...,2023-04-10,5,quiera avisado ibas borrar alv
2,2,2,2,2,William Salgado,2023-04-10 02:59:21,0,De verdad no puedo creerlo sabes algo si ya no...,"('10', '04', '23', '2', '59', '21', 'a.m.', 'W...",2,De verdad no puedo creerlo sabes algo si ya no...,2023-04-10,5,verdad puedo creerlo sabes vas querer verdad
3,3,3,3,3,William Salgado,2023-04-10 03:00:29,0,‎Llamada perdida,"('10', '04', '23', '3', '00', '29', 'a.m.', 'W...",3,‎Llamada perdida,2023-04-10,5,llamada perdida
4,4,4,4,3,Zam• 🧝🏻‍♀️🦋,2023-04-10 03:01:06,0,William estás bien o estás borracho?,"('10', '04', '23', '3', '01', '06', 'a.m.', 'Z...",4,William estás bien o estás borracho?,2023-04-10,5,william bien borracho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10829,11214,11214,10471,13,+52 55 4562 2394,2023-07-03 13:59:00,0,se puede en ambos turnos jaja depende de la di...,"7/3/2023, 1:59 p. m. - +52 55 4562 2394: Se pu...",9,Se puede en ambos turnos jaja depende de la di...,2023-07-03,5,ambos turnos jaja depende disponibilidad
10830,11215,11215,10472,14,+52 55 2179 4145,2023-07-03 14:00:00,0,y dejan su basurero en los salones,"7/3/2023, 2:00 p. m. - +52 55 2179 4145: y dej...",53,y dejan su basurero en los salones,2023-07-03,5,dejan basurero salones
10831,11216,11216,10473,14,+52 55 2179 4145,2023-07-03 14:01:00,0,si hueles,"7/3/2023, 2:01 p. m. - +52 55 2179 4145: Si hu...",54,Si hueles,2023-07-03,5,hueles
10832,11217,11217,10474,14,Pablo(REINA),2023-07-03 14:17:00,0,sera una pequea reunin hoy a las cinco de la t...,"7/3/2023, 2:17 p. m. - Pablo(REINA): Sería una...",42,"Sería una pequeña reunión, hoy a las cinco de ...",2023-07-03,5,pequeña reunión hoy cinco tarde


In [ ]:
df_Pre.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Hour', 'User', 'Date',
       'Day_of_Week', 'Message_Only_Text', 'Message_Raw', 'index',
       'Message_Clean', 'fecha', 'id_conversacion', 'preprocesado'],
      dtype='object')

In [ ]:
df_p = df_Pre.loc[:,['Message_Only_Text']]

## LIMPIEZA

In [ ]:
# Eliminar Puntuacion
df_p['Message_Only_Text'] = df_p['Message_Only_Text'].apply(lambda x: regex_puntuacion.sub('', x))
# Eliminar Acentos
df_p['Message_Only_Text'] = df_p['Message_Only_Text'].apply(lambda x: unidecode(regex_puntuacion.sub('', x)))
# Convertir A minusculas
df_p['Message_Only_Text'] = df_p['Message_Only_Text'].apply(lambda x: x.lower())
# Eliminar StopWorlds el/la/yo etc
df_p['Message_Only_Text'] = df_p['Message_Only_Text'].apply(limpiar_texto)
# Lematizar
df_p['Message_Only_Text'] = df_p['Message_Only_Text'].apply(lematizar)

## Predicciones Etiquetado

In [ ]:
y_pred2 = vectorizer.transform(df_p['Message_Only_Text'])

In [ ]:
y_pred2

<10834x698 sparse matrix of type '<class 'numpy.int64'>'
	with 12722 stored elements in Compressed Sparse Row format>

In [ ]:
y_pred3 = clf.predict(y_pred2)

In [ ]:
y_pred_proba = pd.DataFrame(clf.predict_proba(y_pred2), columns=['proba_clase_0', 'proba_clase_1'])

In [ ]:
df_p['Predicción'] = y_pred3
df_p['Probabilidad No Violento'] = y_pred_proba['proba_clase_0']
df_p['Probabilidad Violento'] = y_pred_proba['proba_clase_1']


In [ ]:
df_p

,Message_Only_Text,Predicción,Probabilidad No Violento,Probabilidad Violento
0,mensaje llamado estar cifrado extremo extremo ...,No violento,0.644027,0.355973
1,si querer avisar ir borrar alv,Violento,0.279692,0.720308
2,verdad poder creer él saber si ir querer si ve...,Violento,0.012738,0.987262
3,llamado perdido,No violento,0.500000,0.500000
4,william bien borracho,Violento,0.414098,0.585902
...,...,...,...,...
10829,poder ambos turno jajo depender disponibilidad,Violento,0.476214,0.523786
10830,dejar basurero salón,Violento,0.308364,0.691636
10831,si holar,No violento,0.566125,0.433875
10832,sera pequeo reunir hoy cinco tarde,No violento,0.738705,0.261295


In [ ]:
df_p.to_csv('datos_lematizados.csv', index=False)

In [ ]:
df_Pre['Etiqueta']=df_p['Predicción']
df_Pre['Probabilidad No Violento'] = y_pred_proba['proba_clase_0']
df_Pre['Probabilidad Violento'] = y_pred_proba['proba_clase_1']

In [ ]:
df_Pre.to_csv('5chats_Etiquetados_Proba.csv', index=False)